# Tutorial - Gaussian Mixture Model with unknown number of components

__Authors__: Feynman Liang <feynmanl@fb.com>, David Noursi <dcalifornia@fb.com>, Walid Taha <wtaha@fb.com>, Nim Arora <nimarora@fb.com>

 The purpose of this tutorial is to introduce some of Bean Machine's support for Gaussian Mixture Processes (GMM) problems.

## Problem

A simple example of an open universe model is a Gaussian Mixture Model (GMM) where the number of mixture components is unknown. 
[Mixture models](https://en.wikipedia.org/wiki/Mixture_model) are useful in problems where individuals from multiple sub-populations are aggregated together. A common use case for GMMs is unsupervised clustering, where one seeks to infer which sub-population an individual belongs without any labeled training data. Using the bean machine open universe probabilistic programming language, we are able to provide a Bayesian treatment of this problem and draw posterior samples representing a distribution over not only the cluster assignments but also the number of clusters itself.


In this tutorial, we will cover (all mathematical symbols defined in "Model" section):

1. How to generate data $\{y_j\}_{j \in [n]}$ from the model, conditioned on the number of clusters $K$ being fixed.
2. Given $\{y_j\}_{j \in [n]}$, how to draw posterior samples of the latent (unobserved) variables $K$, $\mu_i$, $\sigma_i$, and $c_j$

## Prerequisites

The code presented in this tutorial can be executed in the order presented, as long as certain packages are imported as follows:

In [1]:
# unittest
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import torch.distributions as dist
from torch import tensor
import logging

import beanmachine.ppl as bm


logging.getLogger("beanmachine").setLevel(50)
torch.manual_seed(42);

## Model

Concretely, we will consider a GMM where:

 * Number of mixture components $K \sim 1 + \text{Poisson}(2)$ (the $1+$ ensures we have at least one component)
 * Conditioned on $K$, the mixing weights $\alpha \mid K=k \sim \text{Dirichlet}([1, \ldots, 1])$
 * For $i \in [K]$, each mixture component is a 2-dimensional isotropic Gaussian $\mathcal{N}(\mu_i, \sigma_i^2 I)$ where
    * $\mu_i \overset{\text{iid}}{\sim} \mathcal{N}(0, 10I)$
    * $\sigma_i \overset{\text{iid}}{\sim} \text{Gamma}(0.2, 1)$
 * For $j \in [n]$:
     * Conditioned on $\alpha$, the component membership $c_j \mid \alpha \overset{\text{iid}}{\sim} \text{Cat}(\alpha)$
     * Conditioned on component membership, the data $y_j \mid c_j = i \sim \mathcal{N}(\mu_i, \sigma_i^2)$

Our model can be implemented in Bean Machine as follows:

In [2]:
# unittest
class GaussianMixtureModel(object):
    @bm.random_variable
    def K_minus_one(self):
        return dist.Poisson(rate=2.)
        
    @bm.functional
    def K(self):
        return self.K_minus_one() + 1

    @bm.random_variable
    def alpha(self, k):
        return dist.Dirichlet(torch.ones(k))

    @bm.random_variable
    def mu(self, c):
        return dist.MultivariateNormal(loc=torch.zeros(2), covariance_matrix=10.*torch.eye(2))

    @bm.random_variable
    def sigma(self, c):
        return dist.Gamma(1, 1)

    @bm.random_variable
    def component(self, i):
        alpha = self.alpha(self.K().int().item())
        return dist.Categorical(alpha)

    @bm.random_variable
    def y(self, i):
        c = self.component(i).item()
        return dist.MultivariateNormal(loc=self.mu(c), covariance_matrix=self.sigma(c)**2*torch.eye(2))

Notice that `alpha`, which is of dimension `k` conditioned on $K=k$, is parameterized by `k`. This is a current limitation of Bean Machine: any instantiation of a `random_variable` must always have fixed dimension (e.g. `gmm.alpha(3)` is 3-dimensional).

## Observed Sample

Here we generate some observations by conditioning on some latents and running MCMC to sample the prior for the remaining latent variables as well as the data $\{y_j\}_{j \in [n]}$.

In [3]:
n = 12 # num observations
k = 4 # true number of clusters

gmm = GaussianMixtureModel()

ground_truth = {
    **{
        gmm.K_minus_one(): tensor(1.*(k-1)),
        gmm.alpha(k): torch.ones(k)*1./k,
    },
    **{ gmm.mu(i):tensor([1.*int(i/2), i%2]) for i in range(k) },
    **{ gmm.sigma(i):tensor(.1) for i in range(k) },
    **{ gmm.component(i):tensor(i%k) for i in range(n) },
}
# sample latents conditioned on k
queries = (
    [gmm.y(i) for i in range(n)] +
    [gmm.component(i) for i in range(n)] +
    [gmm.mu(i) for i in range(k)] +
    [gmm.sigma(i) for i in range(k)]
)

# sample data using these latents
prior_sample = bm.SingleSiteAncestralMetropolisHastings().infer(
    queries,
    ground_truth,
    num_samples=2,
    num_chains=1)

Samples collected: 100%|██████████| 2/2 [00:00<00:00, 29.21it/s]


Below we visualize the distribution of the generated data points $\{x_j\}_{j \in [n]}$ and use the color to denote the component memberships $\{c_j\}_{j \in [n]}$ of each point. In addition, we display contours at one standard deviation for each of the Gaussians in the mixture.

In [4]:
def draw_points_and_components(ys, components, mus, sigmas):
    df = pd.DataFrame({
        'y0': ys[:,0],
        'y1': ys[:,1],
        'component': components.astype(str),
    }).sort_values(by='component')
    palette = px.colors.qualitative.G10
    fig = px.scatter(df, x='y0', y='y1', color='component', color_discrete_sequence=palette)

    shapes = list()
    _k = mus.shape[0]
    for l in range(_k):
        x0, y0 = mus[l,:] - 2*sigmas[l] # 2 sigma for 95% contour
        x1, y1 = mus[l,:] + 2*sigmas[l]
        shapes.append(dict(
            type="circle",
            xref="x",
            yref="y",
            x0=x0,
            y0=y0,
            x1=x1,
            y1=y1,
            line_color=palette[l]
        ))
    fig.update_layout(shapes=shapes)
    return fig

fig = draw_points_and_components(
    np.vstack([prior_sample.get_variable(gmm.y(i)).squeeze()[-1].detach().numpy() for i in range(n)]),
    np.array([prior_sample.get_variable(gmm.component(i)).squeeze()[-1].item() for i in range(n)]),
    np.vstack([prior_sample.get_variable(gmm.mu(i)).squeeze()[-1].detach().numpy() for i in range(k)]),
    np.array([prior_sample.get_variable(gmm.sigma(i)).squeeze()[-1].item() for i in range(k)]),
)
fig.show()

## Inference

Now we can condition on the observations $\{y_j\}$ and draw samples of the latent variables from the induced posterior.

In [5]:
torch.manual_seed(42)

n_samples = 100

queries = (
    [gmm.K()]
    + [gmm.component(j) for j in range(n)]
    + [gmm.mu(i) for i in range(2*k) ]
    + [gmm.sigma(i) for i in range(2*k)]
)
mh = bm.CompositionalInference()
mh.add_sequential_proposer([
    gmm.K_minus_one,
    gmm.alpha,
    gmm.component,
])
posterior_samples = mh.infer(
    queries,
    {gmm.y(i):prior_sample.get_variable(gmm.y(i)) for i in range(n)},
    num_samples=n_samples,
    num_chains=1)

Samples collected: 100%|██████████| 100/100 [01:39<00:00,  1.00it/s]


## Analysis

Let's first visualise how the number of clusters $K$ varies across a sampling trajectory:

In [6]:
fig = px.line(pd.DataFrame({
    'K': posterior_samples.get_variable(gmm.K()).squeeze().detach().numpy(),
}), y='K')
fig.show()

Next, let's visualize the mixture components (`mu`, `sigma`) and component memberships (`component`) for each of the data points (`y`) at the start of MCMC. This likely won't be very good, since we haven't burned in the chain.

In [7]:
t = 0
components = np.array([posterior_samples.get_variable(gmm.component(i)).squeeze()[t].item() for i in range(n)])
_k = posterior_samples.get_variable(gmm.K()).squeeze()[t].int().item()
fig = draw_points_and_components(
    np.vstack([prior_sample.get_variable(gmm.y(i)).squeeze()[-1].detach().numpy() for i in range(n)]),
    components,
    np.vstack([posterior_samples.get_variable(gmm.mu(i)).squeeze()[t,:].detach().numpy()
               for i in range(_k) if i in np.unique(components)]),
    np.array([posterior_samples.get_variable(gmm.sigma(i)).squeeze()[t].item()
              for i in range(_k) if i in np.unique(components)]),
)
fig.show()

After running `n_samples` MCMC steps, the effects of initialization should be diminished.

In [8]:
t = n_samples - 1
components = np.array([posterior_samples.get_variable(gmm.component(i)).squeeze()[t].item() for i in range(n)])
_k = posterior_samples.get_variable(gmm.K()).squeeze()[t].int().item()
fig = draw_points_and_components(
    np.vstack([prior_sample.get_variable(gmm.y(i)).squeeze()[-1].detach().numpy() for i in range(n)]),
    components,
    np.vstack([posterior_samples.get_variable(gmm.mu(i)).squeeze()[t,:].detach().numpy()
               for i in range(_k) if i in np.unique(components)]),
    np.array([posterior_samples.get_variable(gmm.sigma(i)).squeeze()[t].item()
              for i in range(_k) if i in np.unique(components)]),
)
fig.show()

Plot the components for the sample with the most number of clusters

In [9]:
t = posterior_samples.get_variable(gmm.K()).squeeze().detach().numpy().argmax()
components = np.array([posterior_samples.get_variable(gmm.component(i)).squeeze()[t].item() for i in range(n)])
_k = posterior_samples.get_variable(gmm.K()).squeeze()[t].int().item()
fig = draw_points_and_components(
    np.vstack([prior_sample.get_variable(gmm.y(i)).squeeze()[-1].detach().numpy() for i in range(n)]),
    components,
    np.vstack([posterior_samples.get_variable(gmm.mu(i)).squeeze()[t,:].detach().numpy()
               for i in range(_k) if i in np.unique(components)]),
    np.array([posterior_samples.get_variable(gmm.sigma(i)).squeeze()[t].item()
              for i in range(_k) if i in np.unique(components)]),
)
fig.show()